The goal of this project is to rewrite the pull pipeline we created in the **Application - Pipelines - Pulling** video in the **Generators as Coroutines** section.

You should look at the techniques we used in the **Application - Pipelines - Broadcasting** video and apply them here.

The goal is to write a pipeline that will push data from the source file, `cars.csv`, and push it through some filters and a save coroutine to ultimately save the results as a csv file.

Try to make your code as generic as possible, and don't worry about column headers in the output file (unless you really want to!).

When you are done with your solution you should be able to specify an arbitrary number of filters on the name field.

If you specify `Chevrolet`, `Carlo` and `Landau` for three filters, your output file should contain two lines of data only:

```
Chevrolet Monte Carlo Landau,15.5,8,350.0,170.0,4165.,11.4,77,US
Chevrolet Monte Carlo Landau,19.2,8,305.0,145.0,3425.,13.2,78,US
```

In [93]:
# Create a utility to parse the data from a given .csv file.
import csv

def data_reader(f_name):
    f = open(f_name)
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        yield from csv.reader(f, dialect=dialect)
    finally:
        f.close()

In [94]:
# Create our coroutine decorator to prime coroutines.
def coroutine(fn):
    def inner(*args, **kwargs):
        g = fn(*args, **kwargs)
        next(g)
        return g
    return inner

In [95]:
# Create a data parser to format the rows read from the .csv file
# by the reader utility.
from decimal import Decimal

def data_parser(input_file):
    data = data_reader(input_file)
    headers = [i.lower() for i in next(data)]
    yield headers
    converters = [str, Decimal, int, Decimal, Decimal, lambda x: int(x.strip('.')), Decimal, int, str]
    for row in data:
        parsed_row = [converter(item)
                      for converter, item in zip(converters, row)]
        yield parsed_row

In [96]:
# Create a coroutine to write received rows to a new file.
@coroutine
def save_data(f_name, headers):
    with open(f_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        while True:
            data_row = yield
            writer.writerow(data_row)

In [97]:
def filter_out(rows, keyword):
    for row in rows:
        if keyword in row[0]:
            yield row

In [98]:
def push_data(*keywords):
    # Create data parser.
    data_rows = data_parser('cars.csv')
    # Grab header row.
    headers = next(data_rows)

    for keyword in keywords:
        data_rows = filter_out(data_rows, keyword)
    
    # Set up data file to write out to.
    data_file = save_data('filtered_cars.csv', headers)

    for row in data_rows:
        data_file.send(row)

In [109]:
# push_data('Chevrolet', 'Carlo', 'Landau')
push_data('Chevrolet', 'Chevelle', 'Malibu')

In [110]:
with open('filtered_cars.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['car', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model', 'origin']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504', '12.0', '70', 'US']
['Chevrolet Chevelle Malibu', '17.0', '6', '250.0', '100.0', '3329', '15.5', '71', 'US']
['Chevrolet Chevelle Malibu Classic', '16.0', '6', '250.0', '100.0', '3781', '17.0', '74', 'US']
['Chevrolete Chevelle Malibu', '16.0', '6', '250.0', '105.0', '3897', '18.5', '75', 'US']
['Chevrolet Chevelle Malibu Classic', '17.5', '8', '305.0', '140.0', '4215', '13.0', '76', 'US']
